In [1]:
"""
🔬 EfficientNet-B7 GridSearch - Fair Comparison with Pretrained Weights

EXACT SAME PARAMETERS AS RESNET-101 & EEGNET:
1. ✅ Optimizer: Adam, AdamW, Adagrad
2. ✅ Activation: ReLU, LeakyReLU
3. ✅ L1: [0] (same as others)
4. ✅ L2: [0, 1e-4, 1e-3]
5. ✅ Early Stopping: patience=10
6. ✅ LR Scheduler: CosineAnnealingLR
7. ✅ Loss: SoftFocalLoss (gamma=3.0)
8. ✅ Data: Hybrid loading
9. ✅ CV: 3-fold
10. ✅ Pretrained: ImageNet weights

Total: 18 configs × 3 folds = 54 runs (~15 hours)
"""

import os
from pathlib import Path
import random
import time
import gc
import json
import warnings
from datetime import datetime
from itertools import product
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score



In [2]:
# CELL 1: Setup & Imports


In [3]:

print("="*80)
print(" EfficientNet-B7 GridSearch - Pretrained ".center(80, "="))
print("="*80)
print(f"\nStarted: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
print("✅ Seed: 42")

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

# Paths
DATA_PKG = Path("data_package")
SPEC_DIR = Path("spec_hr_out")
RESULTS_DIR = Path("efficientnet_gridsearch_results")
RESULTS_DIR.mkdir(exist_ok=True)

print(f"\n✅ Results: {RESULTS_DIR}")



=================== EfficientNet-B7 GridSearch - Pretrained ====================

Started: 2026-01-21 17:35:44
✅ Seed: 42
✅ Device: cuda:0
   GPU: NVIDIA GeForce RTX 5060 Ti

✅ Results: efficientnet_gridsearch_results


In [4]:
# CELL 2: Load Data


In [5]:

meta_use = pd.read_csv(DATA_PKG / "meta_use.csv")
lbl = np.load(DATA_PKG / "labels.npz", allow_pickle=True)
y_soft = lbl["y_soft"]
w_conf = lbl["w_conf"]
classes = [str(c) for c in lbl["classes"]]
y_hard = y_soft.argmax(axis=1)

print("✅ Data loaded")
print(f"   Samples: {len(y_hard)}")
print(f"   Classes: {classes}")

# 3-fold CV (SAME AS RESNET & EEGNET & KAN)
N_FOLDS = 3
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
folds = list(skf.split(meta_use, y_hard))
print(f"\n✅ Created {N_FOLDS}-fold CV")



✅ Data loaded
   Samples: 17089
   Classes: ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

✅ Created 3-fold CV


In [6]:
# CELL 3: Dataset Class


In [7]:

class SpecDataset(Dataset):
    def __init__(self, df, root_dir, y_soft, w_conf, F_target=81, T_target=600):
        self.df = df.reset_index(drop=True)
        self.root = Path(root_dir)
        self.y_soft = y_soft
        self.w_conf = w_conf
        self.F_target = F_target
        self.T_target = T_target

    def __len__(self):
        return len(self.df)

    def _center_crop_pad(self, x):
        C, F, T = x.shape
        if F >= self.F_target:
            f0 = (F - self.F_target) // 2
            x = x[:, f0:f0+self.F_target, :]
        else:
            pad = self.F_target - F
            x = np.pad(x, ((0,0),(pad//2, pad-pad//2),(0,0)), mode="constant")
        if T >= self.T_target:
            t0 = (T - self.T_target) // 2
            x = x[:, :, t0:t0+self.T_target]
        else:
            pad = self.T_target - T
            x = np.pad(x, ((0,0),(0,0),(pad//2, pad-pad//2)), mode="constant")
        return x.copy()

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        eid = int(row.eeg_id)
        
        npz = np.load(self.root / f"{eid}_hr.npz")
        x = npz["x"]
        x = self._center_crop_pad(x)
        x = torch.from_numpy(x).float()
        
        # Resize to 224x224 for EfficientNet
        x = F.interpolate(x.unsqueeze(0), size=(224, 224),
                          mode="bilinear", align_corners=False).squeeze(0)
        
        # Convert 4 channels to 3 channels (RGB) for pretrained model
        if x.size(0) == 4:
            # Average first 3 channels + repeat last channel
            x = torch.cat([x[:3].mean(0, keepdim=True).repeat(3, 1, 1)], dim=0)
        
        y = torch.from_numpy(self.y_soft[self.df.index[idx]]).float()
        w = torch.tensor(self.w_conf[self.df.index[idx]], dtype=torch.float32)
        
        return x, y, w

print("✅ Dataset ready")



✅ Dataset ready


In [8]:
# CELL 4: EfficientNet-B7 Model (Pretrained)


In [9]:

from torchvision import models

class EfficientNetB7_Pretrained(nn.Module):
    """EfficientNet-B7 with pretrained ImageNet weights"""
    
    def __init__(self, n_classes=6, activation='relu', freeze_backbone=False):
        super().__init__()
        
        self.activation_name = activation
        
        # Load pretrained EfficientNet-B7
        self.backbone = models.efficientnet_b7(pretrained=True)
        
        # Optionally freeze backbone
        if freeze_backbone:
            for param in self.backbone.features.parameters():
                param.requires_grad = False
        
        # Replace classifier
        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Identity()
        
        # Custom classifier head
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, 512),
            nn.BatchNorm1d(512),
            # Activation will be applied in forward()
            nn.Dropout(0.3),
            nn.Linear(512, n_classes)
        )
    
    def get_activation(self):
        """Return activation function based on config"""
        if self.activation_name == 'relu':
            return F.relu
        elif self.activation_name == 'leakyrelu':
            return lambda x: F.leaky_relu(x, negative_slope=0.01)
        else:
            return F.relu
    
    def forward(self, x):
        # Backbone features
        features = self.backbone(x)
        
        # Classifier with custom activation
        x = self.classifier[0](features)  # Dropout
        x = self.classifier[1](x)  # Linear
        x = self.classifier[2](x)  # BatchNorm
        x = self.get_activation()(x)  # Custom activation
        x = self.classifier[3](x)  # Dropout
        x = self.classifier[4](x)  # Linear
        
        return x

print("✅ EfficientNet-B7 model ready")
print("   Pretrained: ImageNet weights")
print("   Supports: ReLU, LeakyReLU")



✅ EfficientNet-B7 model ready
   Pretrained: ImageNet weights
   Supports: ReLU, LeakyReLU


In [10]:
# CELL 5: SoftFocalLoss


In [11]:

class SoftFocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=3.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, logits, soft_targets, sample_weights=None):
        hard_targets = soft_targets.argmax(dim=1)
        probs = F.softmax(logits, dim=1)
        p_t = probs.gather(1, hard_targets.unsqueeze(1)).squeeze(1)
        ce_loss = -(soft_targets * F.log_softmax(logits, dim=1)).sum(dim=1)
        focal_weight = ((1 - p_t) ** self.gamma)
        loss = focal_weight * ce_loss
        
        if self.alpha is not None:
            alpha_t = self.alpha[hard_targets]
            loss = alpha_t * loss
        
        if sample_weights is not None:
            loss = loss * sample_weights
        
        return loss.mean()

print("✅ SoftFocalLoss ready")



✅ SoftFocalLoss ready


In [12]:
# CELL 6: Hybrid Data Loader


In [13]:

def create_hybrid_loader(fold=0, target_ratio=0.4, weight_power=3.0, batch_size=8):
    tr_idx, va_idx = folds[fold]
    df_tr = meta_use.iloc[tr_idx]
    y_soft_tr, w_conf_tr = y_soft[tr_idx], w_conf[tr_idx]
    
    y_hard = y_soft_tr.argmax(axis=1)
    counts = np.bincount(y_hard, minlength=6)
    target = int(counts.max() * target_ratio)
    
    indices_add = []
    for i in range(6):
        mask = y_hard == i
        if mask.sum() < target:
            idx = np.where(mask)[0]
            n_add = target - mask.sum()
            indices_add.extend(np.random.choice(idx, n_add, replace=True))
    
    all_idx = np.concatenate([np.arange(len(y_hard)), indices_add])
    np.random.shuffle(all_idx)
    
    df_tr_over = df_tr.iloc[all_idx].reset_index(drop=True)
    y_soft_over, w_conf_over = y_soft_tr[all_idx], w_conf_tr[all_idx]
    
    y_hard_over = y_soft_over.argmax(axis=1)
    counts_over = np.bincount(y_hard_over, minlength=6)
    
    weights = (len(y_hard_over) / (counts_over + 1)) ** weight_power
    weights = torch.FloatTensor(weights / weights.sum() * 6)
    
    sample_weights = weights[y_hard_over].numpy()
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    ds_tr = SpecDataset(df_tr_over, SPEC_DIR, y_soft_over, w_conf_over)
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, sampler=sampler, num_workers=0)
    
    ds_va = SpecDataset(meta_use.iloc[va_idx], SPEC_DIR, y_soft[va_idx], w_conf[va_idx])
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=0)
    
    return dl_tr, dl_va, weights

print("✅ Hybrid loader ready (batch_size=8 for B7)")



✅ Hybrid loader ready (batch_size=8 for B7)


In [14]:
# CELL 7: Evaluation


In [15]:

@torch.no_grad()
def evaluate_full(model, loader):
    model.eval()
    preds, targets = [], []
    
    for x, y, w in loader:
        x = x.to(device)
        logits = model(x)
        preds.append(logits.argmax(1).cpu().numpy())
        targets.append(y.argmax(1).cpu().numpy())
    
    y_pred = np.concatenate(preds)
    y_true = np.concatenate(targets)
    
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='macro', zero_division=0),
        'f1': f1_score(y_true, y_pred, average='macro', zero_division=0),
    }

print("✅ Evaluation ready")



✅ Evaluation ready


In [16]:
# CELL 8: Training Function


In [17]:

def train_one_config(fold, optimizer_name, activation, l1_lambda, l2_lambda,
                     lr=3e-4, batch_size=8, epochs=30, patience=10):
    
    # Data
    print(f"      [1/5] Data...", end=" ", flush=True)
    t0 = time.time()
    dl_tr, dl_va, class_weights = create_hybrid_loader(fold=fold, batch_size=batch_size)
    print(f"✓ ({time.time()-t0:.1f}s)", flush=True)
    
    # Model
    print(f"      [2/5] Model (EfficientNet-B7, {activation})...", end=" ", flush=True)
    t0 = time.time()
    model = EfficientNetB7_Pretrained(
        n_classes=6,
        activation=activation,
        freeze_backbone=False  # Fine-tune all layers
    ).to(device)
    print(f"✓ ({time.time()-t0:.1f}s)", flush=True)
    
    # Optimizer
    print(f"      [3/5] Optimizer ({optimizer_name}, L2={l2_lambda:.0e})...", end=" ", flush=True)
    if optimizer_name == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_lambda)
    elif optimizer_name == 'adamw':
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=l2_lambda)
    elif optimizer_name == 'adagrad':
        optimizer = torch.optim.Adagrad(model.parameters(), lr=lr, weight_decay=l2_lambda)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")
    print(f"✓", flush=True)
    
    # Loss & Scheduler
    print(f"      [4/5] Loss & Scheduler...", end=" ", flush=True)
    criterion = SoftFocalLoss(alpha=class_weights.to(device), gamma=3.0)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    print(f"✓", flush=True)
    
    # Training
    print(f"      [5/5] Training (patience={patience}, L1={l1_lambda:.0e})...", flush=True)
    best_f1, best_state, no_improve = 0.0, None, 0
    
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss, n = 0.0, 0
        
        for x, y, w in dl_tr:
            x, y, w = x.to(device), y.to(device), w.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y, w)
            
            # L1 Regularization (SAME AS RESNET & EEGNET)
            if l1_lambda > 0:
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss = loss + l1_lambda * l1_norm
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            train_loss += loss.item() * x.size(0)
            n += x.size(0)
        
        train_loss /= n
        val_results = evaluate_full(model, dl_va)
        scheduler.step()
        
        # Early stopping
        if val_results['f1'] > best_f1:
            best_f1 = val_results['f1']
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"        Early stop at epoch {epoch}", flush=True)
                break
        
        if epoch % 5 == 0 or epoch == 1:
            print(f"        Epoch {epoch:2d}: F1={val_results['f1']:.4f}, Loss={train_loss:.4f}", flush=True)
        
        if epoch % 5 == 0:
            gc.collect()
            torch.cuda.empty_cache()
    
    if best_state:
        model.load_state_dict(best_state)
    
    final_results = evaluate_full(model, dl_va)
    
    del model, optimizer, scheduler, dl_tr, dl_va
    gc.collect()
    torch.cuda.empty_cache()
    
    return final_results

print("✅ Training function ready")
print("   Optimizers: Adam, AdamW, Adagrad")
print("   Activations: ReLU, LeakyReLU")
print("   L1/L2 regularization supported")
print("   Early stopping: patience=10")



✅ Training function ready
   Optimizers: Adam, AdamW, Adagrad
   Activations: ReLU, LeakyReLU
   L1/L2 regularization supported
   Early stopping: patience=10


In [18]:
# CELL 9: Grid Configuration (SAME AS RESNET & EEGNET)


In [19]:

print("\n" + "="*80)
print(" EFFICIENTNET-B7 GRIDSEARCH - SAME AS RESNET-101 ".center(80, "="))
print("="*80)

# EXACT SAME PARAMS AS RESNET-101 & EEGNET
param_grid = {
    'optimizer': ['adam', 'adamw', 'adagrad'],  # 3 - SAME
    'activation': ['relu', 'leakyrelu'],        # 2 - SAME
    'l1_lambda': [0],                           # 1 - SAME (no L1)
    'l2_lambda': [0, 1e-4, 1e-3],              # 3 - SAME
}

fixed_params = {
    'lr': 3e-4,
    'batch_size': 8,  # Smaller batch for B7 memory
    'epochs': 30,
    'patience': 10,
}

keys = list(param_grid.keys())
values = list(param_grid.values())
combinations = list(product(*values))

print("\n📋 HYPERPARAMETER GRID:")
print("-"*80)
print(f"  Optimizer:   {param_grid['optimizer']}")
print(f"  Activation:  {param_grid['activation']}")
print(f"  L1 lambda:   {param_grid['l1_lambda']}")
print(f"  L2 lambda:   {param_grid['l2_lambda']}")

print("\n📊 GRIDSEARCH STATISTICS:")
print("-"*80)
print(f"  Total combinations: {len(combinations)}")
print(f"  Folds per config:   {N_FOLDS}")
print(f"  Total trainings:    {len(combinations) * N_FOLDS}")
print(f"  Est. time per run:  ~15-20 min (B7 is larger)")
print(f"  Est. total time:    ~{len(combinations) * N_FOLDS * 17.5 / 60:.1f} hours")

print("\n📝 ALL COMBINATIONS:")
print("-"*80)
for i, combo in enumerate(combinations, 1):
    params = dict(zip(keys, combo))
    print(f"  {i:2d}. {params['optimizer']:7s} + {params['activation']:10s} + "
          f"L1={params['l1_lambda']:.0e} + L2={params['l2_lambda']:.0e}")

print("\n🎯 TARGETS TO BEAT:")
print("-"*80)
print("  EEGNet (baseline):     F1 = 0.3281")
print("  EEGNet (tuned):        F1 = 0.3892")
print("  ResNet-101 (tuned):    F1 = 0.5585")
print("  EfficientNet-B7:       F1 = ??? (GOAL: > 0.5585)")

print("\n⏱️  TIMELINE:")
print("-"*80)
current_time = datetime.now()
finish_time = current_time + pd.Timedelta(hours=len(combinations) * N_FOLDS * 17.5 / 60)
print(f"  Start:  {current_time.strftime('%Y-%m-%d %H:%M')}")
print(f"  Finish: {finish_time.strftime('%Y-%m-%d %H:%M')} (approx)")

print("\n💾 AUTO-SAVE:")
print("-"*80)
print(f"  {RESULTS_DIR}/efficientnet_gridsearch_progress.json")
print(f"  {RESULTS_DIR}/efficientnet_gridsearch_final.json")




=============== EFFICIENTNET-B7 GRIDSEARCH - SAME AS RESNET-101 ================

📋 HYPERPARAMETER GRID:
--------------------------------------------------------------------------------
  Optimizer:   ['adam', 'adamw', 'adagrad']
  Activation:  ['relu', 'leakyrelu']
  L1 lambda:   [0]
  L2 lambda:   [0, 0.0001, 0.001]

📊 GRIDSEARCH STATISTICS:
--------------------------------------------------------------------------------
  Total combinations: 18
  Folds per config:   3
  Total trainings:    54
  Est. time per run:  ~15-20 min (B7 is larger)
  Est. total time:    ~15.8 hours

📝 ALL COMBINATIONS:
--------------------------------------------------------------------------------
   1. adam    + relu       + L1=0e+00 + L2=0e+00
   2. adam    + relu       + L1=0e+00 + L2=1e-04
   3. adam    + relu       + L1=0e+00 + L2=1e-03
   4. adam    + leakyrelu  + L1=0e+00 + L2=0e+00
   5. adam    + leakyrelu  + L1=0e+00 + L2=1e-04
   6. adam    + leakyrelu  + L1=0e+00 + L2=1e-03
   7. adamw   + relu

In [20]:
# CELL 10: Run GridSearch


In [21]:

print("\n" + "="*80)
print(" STARTING GRIDSEARCH ".center(80, "="))
print("="*80)
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

all_results = []
start_time = time.time()

for combo_idx, combo in enumerate(combinations, 1):
    params = dict(zip(keys, combo))
    
    print("\n" + "="*80)
    print(f" CONFIG {combo_idx}/{len(combinations)} ".center(80, "="))
    print("="*80)
    print(f"  Optimizer: {params['optimizer']}")
    print(f"  Activation: {params['activation']}")
    print(f"  L1: {params['l1_lambda']:.0e}")
    print(f"  L2: {params['l2_lambda']:.0e}")
    print("-"*80)
    
    fold_results = []
    
    for fold in range(N_FOLDS):
        print(f"\n    Fold {fold+1}/{N_FOLDS}...", flush=True)
        fold_start = time.time()
        
        try:
            result = train_one_config(
                fold=fold,
                optimizer_name=params['optimizer'],
                activation=params['activation'],
                l1_lambda=params['l1_lambda'],
                l2_lambda=params['l2_lambda'],
                **fixed_params
            )
            fold_results.append(result)
            print(f"\n    ✓ Fold {fold+1}: F1={result['f1']:.4f} ({(time.time()-fold_start)/60:.1f} min)", flush=True)
        except Exception as e:
            print(f"\n    ✗ Error: {e}", flush=True)
            fold_results.append({'f1': 0.0, 'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0})
    
    mean_metrics = {
        'f1': np.mean([r['f1'] for r in fold_results]),
        'accuracy': np.mean([r['accuracy'] for r in fold_results]),
        'precision': np.mean([r['precision'] for r in fold_results]),
        'recall': np.mean([r['recall'] for r in fold_results]),
        'f1_std': np.std([r['f1'] for r in fold_results]),
    }
    
    result_entry = {
        'config_id': combo_idx,
        'params': params,
        'mean_metrics': mean_metrics,
        'fold_results': fold_results,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    }
    all_results.append(result_entry)
    
    print(f"\n  Mean F1: {mean_metrics['f1']:.4f} ± {mean_metrics['f1_std']:.4f}")
    print(f"  Mean Acc: {mean_metrics['accuracy']:.4f}")
    
    # Auto-save
    with open(RESULTS_DIR / 'efficientnet_gridsearch_progress.json', 'w') as f:
        json.dump(all_results, f, indent=2, default=str)
    print(f"  💾 Saved", flush=True)

# Final save
with open(RESULTS_DIR / 'efficientnet_gridsearch_final.json', 'w') as f:
    json.dump({
        'all_results': all_results,
        'param_grid': param_grid,
        'fixed_params': fixed_params,
        'total_time_hours': (time.time() - start_time) / 3600,
    }, f, indent=2, default=str)

print("\n" + "="*80)
print(" GRIDSEARCH COMPLETE ".center(80, "="))
print("="*80)
print(f"Total time: {(time.time()-start_time)/3600:.2f} hours")




============================= STARTING GRIDSEARCH ==============================
Started: 2026-01-21 17:45:06


================================= CONFIG 1/18 ==================================
  Optimizer: adam
  Activation: relu
  L1: 0e+00
  L2: 0e+00
--------------------------------------------------------------------------------

    Fold 1/3...
      [1/5] Data... ✓ (0.0s)
      [2/5] Model (EfficientNet-B7, relu)... Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to C:\Users\numpppy/.cache\torch\hub\checkpoints\efficientnet_b7_lukemelas-c5b4e57e.pth


100%|██████████| 255M/255M [00:22<00:00, 11.7MB/s] 


✓ (24.1s)
      [3/5] Optimizer (adam, L2=0e+00)... ✓
      [4/5] Loss & Scheduler... ✓
      [5/5] Training (patience=10, L1=0e+00)...
        Epoch  1: F1=0.2963, Loss=0.8956
        Epoch  5: F1=0.3420, Loss=0.2265
        Epoch 10: F1=0.3540, Loss=0.1220
        Epoch 15: F1=0.4210, Loss=0.0980
        Epoch 20: F1=0.4847, Loss=0.0815
        Epoch 25: F1=0.5171, Loss=0.0809
        Epoch 30: F1=0.5180, Loss=0.0714

    ✓ Fold 1: F1=0.5181 (174.8 min)

    Fold 2/3...
      [1/5] Data... ✓ (0.0s)
      [2/5] Model (EfficientNet-B7, relu)... ✓ (0.7s)
      [3/5] Optimizer (adam, L2=0e+00)... ✓
      [4/5] Loss & Scheduler... ✓
      [5/5] Training (patience=10, L1=0e+00)...
        Epoch  1: F1=0.3086, Loss=0.8989
        Epoch  5: F1=0.3585, Loss=0.2052
        Epoch 10: F1=0.3621, Loss=0.1054
        Epoch 15: F1=0.4007, Loss=0.0988
        Epoch 20: F1=0.4930, Loss=0.0800
        Epoch 25: F1=0.5147, Loss=0.0696
        Epoch 30: F1=0.5199, Loss=0.0757

    ✓ Fold 2: F1=0.5213 (1

In [ ]:
# CELL 11: Analyze Results


In [23]:
# ========================================================================
# CELL 11: Analyze Results (WITH PRECISION & RECALL)
# ========================================================================

sorted_results = sorted(all_results, key=lambda x: x['mean_metrics']['f1'], reverse=True)

print("\n" + "="*80)
print(" EFFICIENTNET-B7 GRIDSEARCH RESULTS ".center(80, "="))
print("="*80)

print("\n🏆 TOP 10 CONFIGURATIONS:")
print("="*80)
print(f"{'Rank':<6} {'Optimizer':>10} {'Activation':>12} {'L1':>8} {'L2':>8} "
      f"{'F1':>10} {'Acc':>8} {'Prec':>8} {'Rec':>8}")
print("-"*80)

for i, result in enumerate(sorted_results[:10], 1):
    p = result['params']
    m = result['mean_metrics']
    print(f"{i:<6} {p['optimizer']:>10} {p['activation']:>12} {p['l1_lambda']:>8.0e} "
          f"{p['l2_lambda']:>8.0e} {m['f1']:>10.4f} {m['accuracy']:>8.4f} "
          f"{m['precision']:>8.4f} {m['recall']:>8.4f}")

# Best config
best = sorted_results[0]

print("\n" + "="*80)
print(" BEST EFFICIENTNET-B7 CONFIGURATION ".center(80, "="))
print("="*80)
print(f"  Optimizer:  {best['params']['optimizer']}")
print(f"  Activation: {best['params']['activation']}")
print(f"  L1:         {best['params']['l1_lambda']:.0e}")
print(f"  L2:         {best['params']['l2_lambda']:.0e}")

print(f"\n  F1:        {best['mean_metrics']['f1']:.4f} ± {best['mean_metrics']['f1_std']:.4f}")
print(f"  Accuracy:  {best['mean_metrics']['accuracy']:.4f}")
print(f"  Precision: {best['mean_metrics']['precision']:.4f}")
print(f"  Recall:    {best['mean_metrics']['recall']:.4f}")

# Final comparison
print("\n" + "="*80)
print(" FINAL COMPARISON ".center(80, "="))
print("="*80)
print(f"  EEGNet (baseline):     F1 = 0.3281,  Acc = 0.3154")
print(f"  EEGNet (tuned):        F1 = 0.3892,  Acc = 0.3731")
print(f"  ResNet-101 (tuned):    F1 = 0.5585,  Acc = 0.5921")
print(f"  EfficientNet-B7:       F1 = {best['mean_metrics']['f1']:.4f}, "
      f"Acc = {best['mean_metrics']['accuracy']:.4f}, "
      f"Prec = {best['mean_metrics']['precision']:.4f}, "
      f"Rec = {best['mean_metrics']['recall']:.4f}")

# Verdict
if best['mean_metrics']['f1'] > 0.5585:
    improvement = ((best['mean_metrics']['f1'] - 0.5585) / 0.5585) * 100
    print(f"\n  ✅ EfficientNet-B7 BEATS ResNet-101 by {improvement:.1f}%!")
elif best['mean_metrics']['f1'] > 0.3892:
    print(f"\n  ✅ EfficientNet-B7 beats EEGNet but loses to ResNet-101")
else:
    print(f"\n  ⚠️  EfficientNet-B7 underperformed")

print("\n💾 Results saved to:")
print(f"   {RESULTS_DIR}/efficientnet_gridsearch_final.json")


====================== EFFICIENTNET-B7 GRIDSEARCH RESULTS ======================

🏆 TOP 10 CONFIGURATIONS:
Rank    Optimizer   Activation       L1       L2         F1      Acc     Prec      Rec
--------------------------------------------------------------------------------
1           adamw         relu    0e+00    1e-03     0.3475   0.3922   0.3582   0.3411
2           adamw         relu    0e+00    1e-04     0.3472   0.3847   0.3594   0.3452
3            adam    leakyrelu    0e+00    0e+00     0.3472   0.3890   0.3544   0.3424
4            adam         relu    0e+00    0e+00     0.3465   0.3923   0.3626   0.3364
5           adamw    leakyrelu    0e+00    1e-03     0.3446   0.3891   0.3538   0.3385
6           adamw    leakyrelu    0e+00    1e-04     0.3417   0.3871   0.3486   0.3395
7           adamw    leakyrelu    0e+00    0e+00     0.3413   0.3819   0.3477   0.3378
8           adamw         relu    0e+00    0e+00     0.3400   0.3871   0.3507   0.3332
9            adam    leakyre